In [ ]:
import bs4 as bs
import urllib.request
import pickle
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import numpy as np

In [ ]:
#scraping 

def scraper(x,y,proxy):
    
    global all

    for yr in range(x,y,1):
        site=f'http://stats.espncricinfo.com/ci/engine/records/averages/batting.html?class=2;id={yr};type=year'
        response = requests.get(site,proxies={"http": proxy})

        soup=bs.BeautifulSoup(response.text,'lxml')
        #sauce=urllib.request.urlopen(site).read()
        #soup=bs.BeautifulSoup(sauce,'lxml')

        table=soup.find('table')
        #print(table)
        table_rows=soup.find_all('tr')
        #odi=[]


        for tr in table_rows: 
            td=tr.find_all('td')

            rows=[i.text for i in td if i.text[0]!='\n']
            #print(rows)
            #print(len(rows))
            if len(rows)!=0:
                #print(rows[0])
                players=rows[0]
                runs=rows[4]
                if runs!='-':
                    #print(players,runs)
                    all.append((yr,players,runs))

        time.sleep(2)




#df=pd.DataFrame(odi,columns=columns)
#df.head()

In [ ]:
%%time

from lxml.html import fromstring
import requests
from itertools import cycle
import traceback


def get_proxies():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    parser = fromstring(response.text)
    proxies = set()
    for i in parser.xpath('//tbody/tr')[:10]:
        if i.xpath('.//td[7][contains(text(),"no")]'):
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            proxies.add(proxy)
    return proxies


proxies = get_proxies()
proxy_pool = cycle(proxies)
l=1971
columns=['Year','Player', 'Runs']
all=[]
for i in range(1,11):
    #Get a proxy from the pool
    proxy = next(proxy_pool)
    print("Request #%d"%i)
    u=l+4
    try:
        #response = requests.get(url,proxies={"http": proxy, "https": proxy})
        #print(response.json())
        if u<=2020:    
            print(l,u)
            scraper(l,u,proxy)
            l=u
    except:
            print("Skipping. Connnection error")
            #i=i-1

df=pd.DataFrame(all,columns=columns)        
df.to_csv('odi.csv',index=False)

